preprocessing

In [ ]:
import cv2
import numpy as np
import os
from mtcnn import MTCNN
import albumentations as A

2024-06-12 20:38:56.728649: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 20:38:58.012632: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
detector = MTCNN()

In [ ]:
augmentation_pipeline = A.Compose([
    A.GaussNoise(p=0.2),
    A.GaussianBlur(p=0.2),
    A.HorizontalFlip(p=0.5),
    A.Resize(384, 384, interpolation=cv2.INTER_LINEAR),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.5)
])

In [ ]:
def preprocess_image(image_path):
    # Load image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Detect faces using MTCNN
    faces = detector.detect_faces(image_rgb)
    if len(faces) == 0:
        return None
    
    # Extract the first face detected
    x, y, width, height = faces[0]['box']
    face = image_rgb[y:y+height, x:x+width]
    
    # Apply augmentation
    augmented = augmentation_pipeline(image=face)
    face_augmented = augmented['image']
    
    # Convert back to BGR for consistency
    face_bgr = cv2.cvtColor(face_augmented, cv2.COLOR_RGB2BGR)
    
    return face_bgr


In [ ]:
dataset_path = '/home/shree_xd/Documents/deepfake_detection/'
output_path = '/home/shree_xd/Documents/deepfake_detection/preprocessed/'


In [ ]:
# Create the output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

In [ ]:
for category in ['real', 'fake']:
    category_path = os.path.join(dataset_path, category)
    for filename in os.listdir(category_path):
        file_path = os.path.join(category_path, filename)
        preprocessed_image = preprocess_image(file_path)
        if preprocessed_image is not None:
            output_file_path = os.path.join(output_path, f"{category}_{filename}")
            cv2.imwrite(output_file_path, preprocessed_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [ ]:
def load_data(preprocessed_path, labels_file):
    images = []
    labels = []

    with open(labels_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            filename, label = line.strip().split(',')
            img_path = os.path.join(preprocessed_path, filename)
            if os.path.exists(img_path):
                image = tf.keras.preprocessing.image.load_img(img_path)
                image = tf.keras.preprocessing.image.img_to_array(image)
                images.append(image)
                labels.append(int(label))
    
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

In [ ]:
preprocessed_path = '/home/shree_xd/Documents/deepfake_detection/preprocessed'
labels_file = '//home/shree_xd/Documents/deepfake_detection/dataset/labels.csv'


In [ ]:
images, labels = load_data(preprocessed_path, labels_file)


: 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
